In [ ]:
import pandas as pd
import numpy as np

energy_data = pd.read_csv('data.csv')
energy_data

In [ ]:
energy_data = energy_data[energy_data['ENERGY SOURCE'] != 'Pumped Storage'] # Exclude pumped storage, as its stored energy

In [ ]:
grouped_data = energy_data.groupby(['YEAR', 'MONTH', 'STATE', 'ENERGY SOURCE'])['GENERATION (Megawatthours)'].sum().reset_index()

filtered_data = grouped_data[~grouped_data['ENERGY SOURCE'].str.contains("Total")]

total_generation = filtered_data.groupby(['YEAR', 'MONTH', 'STATE'])['GENERATION (Megawatthours)'].sum().reset_index()
total_generation.rename(columns={'GENERATION (Megawatthours)': 'TOTAL GENERATION'}, inplace=True)

merged_data = pd.merge(filtered_data, total_generation, on=['YEAR', 'MONTH', 'STATE'])

merged_data['PERCENTAGE'] = (merged_data['GENERATION (Megawatthours)'] / merged_data['TOTAL GENERATION']) * 100

merged_data.head()

In [ ]:
import json

merged_data['PERCENTAGE'].fillna(0, inplace=True) # DC sometimes generated no power for some months, leading to NaN values

# Mapping state name to expected ID by D3 Choropleth
state_to_id = {
    'AK': '02', 'AL': '01', 'AR': '05', 'AZ': '04', 'CA': '06',
    'CO': '08', 'CT': '09', 'DC': '11', 'DE': '10', 'FL': '12',
    'GA': '13', 'HI': '15', 'IA': '19', 'ID': '16', 'IL': '17',
    'IN': '18', 'KS': '20', 'KY': '21', 'LA': '22', 'MA': '25',
    'MD': '24', 'ME': '23', 'MI': '26', 'MN': '27', 'MO': '29',
    'MS': '28', 'MT': '30', 'NC': '37', 'ND': '38', 'NE': '31',
    'NH': '33', 'NJ': '34', 'NM': '35', 'NV': '32', 'NY': '36',
    'OH': '39', 'OK': '40', 'OR': '41', 'PA': '42', 'RI': '44',
    'SC': '45', 'SD': '46', 'TN': '47', 'TX': '48', 'UT': '49',
    'VA': '51', 'VT': '50', 'WA': '53', 'WI': '55', 'WV': '54',
    'WY': '56', 'US-TOTAL': "00"
}

json_output = []
for (year, month), group in merged_data.groupby(['YEAR', 'MONTH']):
    month_data = []
    for state, state_group in group.groupby('STATE'):
        energy_mix = {row['ENERGY SOURCE']: round(float(row['PERCENTAGE']), 2) for index, row in state_group.iterrows()}
        month_data.append({
            "id": state_to_id[state],
            "EnergyMix": energy_mix
        })
    json_output.append({
        "Year": int(year),
        "Month": int(month),
        "Data": month_data
    })

json_formatted_str = json.dumps(json_output, indent=2)
json_formatted_str[:1000]

In [ ]:
with open('../src/assets/data.json', 'w') as f:
    f.write(json_formatted_str)